# Lecture 2: Solutions

In [1]:
import numpy as np

In [2]:
# Change numpy's printoptions: we don't need to see so many digits or scientific notation.
np.set_printoptions(precision=4, suppress=True)

## Distance matrix

Just as for Matlab (as we will see later), performing mathematical operations on whole arrays using `numpy` functions is often much faster than doing many individual operations in a loop. Let's compute a distance matrix for $m$ points in $n$ dimensional space given in a matrix $X$. We need to compute the $m \times m$ matrix $D$ such that
$$D_{ij} = \vert| x_i - x_j \vert|_2,$$
where $x_i$ and $x_j$ are the $i$th and $j$th row of $m \times n$ matrix $X$, respectively. 

Remember that for a vector $y = (y(1),y(2),\dots,y(n)) \in \mathbb{R}^n$,  

$$
\vert| y \vert|_2 = \sqrt{\sum_{i = 1}^n y(i)^2}
$$


1. Implement a function computing the distance matrix using two loops.
2. Implement a function computing the distance matrix using one loop.
3. Test your implementations for a small random matrix, say $6\times 4$, by comparing the result with the result obtained by the `distance_matrix` function from the `scipy.spatial` module. Note that you have to call this function with two arguments, e.g., `distance_matrix(x, x)`.
4. Test the performance of the three methods on a large matrix, say $1000 \times 100$ using iPythons magic command `%timeit`.

<i> In Spyder, the use of `%timeit` yields a warning but the command should work (this might take a moment though, as `%timeit` takes some time to run). You can avoid the warning by storing your code for this exercise using the `.ipy` extension. I did not (yet) manage to make `%timeit` work in Visual Studio Code...</i>

In [3]:
from scipy.spatial import distance_matrix

In [4]:
def distance_two_loops(x):
    """Compute Euclidean distance matrix using two loops."""
    m = x.shape[0]
    result = np.zeros((m, m))
    for i in range(m - 1):
        for j in range(i + 1, m):
            # Distance between rows i and j
            row_diff = x[i] - x[j]
            result[i, j] = np.sqrt(np.sum(row_diff**2))
            
            # Use symmetry.
            result[j, i] = result[i, j]
    return result

def distance_one_loop(x):
    """Compute Euclidean distance matrix using one loop."""
    m = x.shape[0]
    result = np.zeros((m, m))
    for i in range(m - 1):
        # Difference of row i and all rows below.
        row_diff = x[i] - x[i+1:]
        result[i+1:, i] = np.sqrt(np.sum(row_diff**2, axis=1))
        
        # Use symmetry.
        result[i, i+1:] = result[i+1:, i]
    return result

In [5]:
# Create a small test matrix
m, n = 6, 4
x = np.random.randn(m, n)
x

array([[-3.6828,  0.3497, -0.1823, -2.3487],
       [-0.0948, -0.8232, -0.4258, -0.9335],
       [ 1.8059,  1.0072, -0.6578,  0.5216],
       [ 0.3649,  0.1568, -1.5085,  0.2901],
       [ 0.2438,  0.3925,  0.8236, -0.3002],
       [ 0.7122, -0.6219, -0.09  , -1.3007]])

In [6]:
%timeit distance_two_loops(x)

99 µs ± 4.26 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
%timeit distance_one_loop(x)

43.5 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
%timeit distance_matrix(x,x)

16.7 µs ± 622 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Cramer's rule

**Cramer's rule** is a theorem from linear algebra for the solution of a square linear system of equations $Ax=b$. Have a quick look at the [Wikipedia](https://en.wikipedia.org/wiki/Cramer%27s_rule) page about Cramer's rule or check your linear algebra textbook. Now implement your own implementation of Cramer's rule such that you can use it to solve a square linear system by calling `cramer(A, b)`. You can use numpy's `det` function to compute the determinant of a matrix.

Test your method on some randomly generated matrix $A$ and vector $b$ and verify the correctness of your solution either by directly verifying the equality or by comparing the solution obtained by `numpy.linalg.solve`. You might want to use `np.allclose` to discard numerical inaccuracies.

In [10]:
def cramer(a, b):
    """Solve square linear system using Cramer's rule.
    
    Compute the solution `x` of the square linear system of equations
        ax = b
    
    Arguments
    ---------
        a : (n, n) ndarray
        b : (n,) ndarray
    """
    m, n = a.shape
    if m != n:
        raise ValueError('Matrix must be square.')
    x = np.empty(m) #Alternatively, you can, for example, use x = np.zeros(m)
    # Compute determinant after replacing i-th column by `b`.
    for i in range(m):
        a_mod = a.copy()
        a_mod[:, i] = b
        x[i] = np.linalg.det(a_mod)
    return x / np.linalg.det(a)

In [11]:
# Test Cramer's rule.
n = 4
a = np.random.randn(n, n)
b = np.random.randn(n)

x = np.linalg.solve(a, b)
y = cramer(a, b)

# Verify correctness (both tests are sufficient).
if np.allclose(x, y) and np.allclose(a @ y, b):
    print('Result okay')
else:
    print("Something's wrong")

Result okay


## Obtaining different solutions to a under-determined linear system
Create a matrix $A \in \mathbb{R}^{6 \times 10}$, and and a vector $b \in \mathbb{R}^6$ both with standard normal random numbers. 

1. Check that the matrix has full rank (i.e., 6); if not generate new standard normal random numbers. 
2. Obtain an arbitrary particular solution $\tilde{x}$ for the linear system $Ax=b$.
3. Find the particular solution $x\in\mathbb{R}^{10}$ for which the first four elements are equal to zero. Rather than using the *null space* of $A$, just discard the first four columns of $A$ and solve the remaining 6-by-6 system of linear equations.
4. Verify that the vectors you found in (1) and (2) are actual solutions to the system $Ax=b$.

In [17]:
# Create some random data.
m, n = 6, 10
A = np.random.randn(m, n)
b = np.random.randn(m)
rank = np.linalg.matrix_rank(A)
print(f'Matrix rank: {rank}')  #or print('Matrix rank: %i' % (rank))

Matrix rank: 6


In [18]:
# Solve underdetermined system using lstsq.
xp, *_ = np.linalg.lstsq(A, b, rcond=None)

In [19]:
# Discard first four columns of `a` and solve 6-by-6 linear system using `solve`.
x_sub = np.linalg.solve(A[:, -m:], b)

# Create the 10-dimensional solution by prepending four zeros.
x_zeros = np.concatenate((np.zeros(n - m), x_sub))

In [20]:
# Put the two solutions in columns of matrix.
X = np.stack((xp, x_zeros),axis=1) #Or use np.vstack/np.hstack
print(X)

[[ 0.3619  0.    ]
 [ 0.0136  0.    ]
 [ 0.0887  0.    ]
 [ 0.1581  0.    ]
 [ 0.2404  1.2059]
 [-0.1676 -0.266 ]
 [-0.2801 -0.2281]
 [-0.043   1.7866]
 [-0.0903 -0.3628]
 [ 0.0619 -0.1968]]


In [23]:
print(np.allclose(A@xp,b))
print(np.allclose(A@x_zeros,b))

True
True


## Finding integral minimum of function
Consider the polynomial function $$g(x,y,z) = \left(x - \frac{3}{2}\right)\left(y - \frac{21}{8}\right)\left(z + \frac{4}{3}\right).$$ 

1. Write a function that computes the value

    $$
    v = \min \{g(x,y,z) : (x,y,z) \in [a,b] \times [c,d] \times [e,f] \cap \mathbb{Z}^3\}
    $$
    
    where $a,b,c,d,e,f \in \mathbb{Z}$, i.e., these numbers are (possibly negative) integers. <i>That is, we want to compute the minimal function value of $g$ amongst all integral points in the box $[a,b] \times [c,d] \times [e,f]$.</i>

2. Test your function for the box $[-5,5]^3$, i.e., $a = c = e = -5$ and $b = d = f = 5$.

In [26]:
#Write the function here
import itertools as it

def g(x):
    """ This function computes the function value of g in the point x
    
    Arguments
    ---------
        x       : The desired point in which function is evaluated.
    """
    return (x[0] - 3/2)*(x[1]-21/8)*(x[2] + 4/3)

def box_minimum(g,a,b,c,d,e,f):
    """This functions computes the minimum value of a function over a specified box domain
    
    Arguments
    ---------
        g       : The function over which is optimised (should be R^3 -> R)
        a,...,f : Specify box domain
    """
    function_values = np.array([g(np.array(x)) for x in it.product(range(a,b+1),range(c,d+1),range(e,f+1))])
    return np.min(function_values)

In [27]:
#Test the function here
a,c,e = -5,-5,-5
b,d,f = 5,5,5
y = box_minimum(g,a,b,c,d,e,f)
print(y)

-181.72916666666669


Now suppose that, for given $a_1,\dots,a_n$, we have a polynomial of the form 
$$
g(x_1,\dots,x_n) = \prod_{i=1}^n (x - a_i).
$$
1. Write a function that computes

    $$
    \min \{g(x_1,\dots,x_n) : (x_1,\dots,x_n) \in [-r,r]^n \cap \mathbb{Z}^n\} 
    $$
    
    for $r \in \mathbb{N}$. The inputs of this function are the array $a = [a_1,\dots,a_n]$ and the parameter $r$. 
    If no value for $r$ is given, have the functions set it to $r = 1$ by default. 
1. Test your function for $r = 5$ and the vector $a = (3/2, 21/8,-4/3)$ (the values of the first part of this exercise).

In [28]:
#Write the function here
def g(x,a):
    return np.prod(x-a)
        
def ndim_box_minimum(a,r=1):    
    n = np.size(a)
    function_values = np.array([g(np.array(x),a) for x in it.product(range(-r,r+1),repeat=n)])
    return np.min(function_values)

In [29]:
#Test the function here
a = np.array([3/2,21/8,-4/3])
r = 5
ndim_box_minimum(a,r)

-181.72916666666669